# 1. 零件清单 (Core Components)

在这一步，我们要实现以下四个核心组件：

1. 词表 (Vocabulary)：模拟一个简单的映射。

2. 词嵌入 (Embedding)：将数字映射为稠密向量。

3. 位置编码 (Positional Encoding)：让模型知道词的顺序。

4. 组装核心输入层：将 Embedding 和 Position 相加。

我们将使用 PyTorch 的 nn.Module 来构建这些。

## 第一步：环境准备与参数定义

首先，我们定义一些基础的“超参数”。

In [1]:
import torch
import torch.nn as nn
import math

# 超参数定义
vocab_size = 1000  # 词表大小（假设有1000个常用词）
d_model = 512      # 每个单词身份证（向量）的长度
max_len = 100      # 句子最大长度
n_heads = 3        # 你要求的3个注意力头

## 第二步：位置编码 (Positional Encoding)

这是 Transformer 最“数学”的地方。我们使用正弦和余弦函数的组合，这样模型可以通过相对位置来学习顺序。

In [2]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        
        # 创建一个 [max_len, d_model] 的矩阵来存放位置编码
        pe = torch.zeros(max_len, d_model)
        
        # position 是 [0, 1, 2, ..., max_len-1] 的列向量
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        # 计算分母部分（利用 log 空间计算保证数值稳定性）
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # 偶数位置用 sin，奇数位置用 cos
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # 增加一个 batch 维度变成 [1, max_len, d_model]
        pe = pe.unsqueeze(0)
        
        # register_buffer 的作用是：这个张量是模型的一部分，但不参与梯度更新
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x 的维度是 [batch_size, seq_len, d_model]
        # 我们把对应的位置编码加到 x 上
        x = x + self.pe[:, :x.size(1), :]
        return x

## 第三步：组装零件清单 (Input Layer)

现在我们把 Embedding 和 Positional Encoding 封装在一起。

In [3]:
class TransformerInput(nn.Module):
    def __init__(self, vocab_size, d_model, max_len):
        super().__init__()
        # 1. 词嵌入层：把 ID 变成 512 维向量
        self.embedding = nn.Embedding(vocab_size, d_model)
        
        # 2. 位置编码层
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        
        # 3. Dropout 层：训练中随机丢弃一部分神经元，防止过拟合
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        # x (词 ID): [batch_size, seq_len]
        
        # 第一步：ID 变向量
        x = self.embedding(x)  # 输出: [batch_size, seq_len, d_model]
        
        # 第二步：加上位置信息
        x = self.pos_encoding(x)
        
        # 第三步：Dropout
        return self.dropout(x)

## 验证一下这部分代码
我们可以运行一段测试代码，看看输入 "I love AI" 后的变化：

In [4]:
# 模拟输入：1行句子，包含3个单词的 ID
dummy_input = torch.tensor([[12, 45, 88]]) 

# 初始化输入层
input_layer = TransformerInput(vocab_size, d_model, max_len)

# 运行前向传播
output = input_layer(dummy_input)

print(f"输入 ID 的形状: {dummy_input.shape}")      # torch.Size([1, 3])
print(f"经过核心零件后的形状: {output.shape}")    # torch.Size([1, 3, 512])
print("第一部分的零件已经成功运行！")

输入 ID 的形状: torch.Size([1, 3])
经过核心零件后的形状: torch.Size([1, 3, 512])
第一部分的零件已经成功运行！
